In [57]:
import pandas as pd
import numpy as np

In [58]:
from yahooquery import Ticker
def get_data(ticker, start = 0, end = 0, period = -1):
    stock = Ticker(ticker)
    if period == 'max':
        df = stock.history(period='max')
        
    else:
        df = stock.history(start=start, end=end)
    
    df = df.xs(ticker)
    return df

# Creating a portfolio

In [59]:
import random

In [60]:
ibov = pd.read_csv('IBOVDia_03-08-21 (1).csv')

In [61]:
stocks = ibov.iloc[:,0].values

In [62]:
def create_portfolio():
    portfolio_size = random.randint(3,10)#number of stocks in this portfolio

    portfolio_stocks = random.sample(list(stocks), portfolio_size) #portfolio_composition

    weights = np.random.random(portfolio_size)
    weights /= weights.sum() #portfolio weights

    df = get_data(portfolio_stocks[0]+'.SA', start='2020-03-03', end='2021-07-31')[['close']] * weights[0] #creating the dataframe
    for (stock, weight) in zip(portfolio_stocks[1:], weights[1:]):
        data = get_data(stock+'.SA', start='2020-03-03', end='2021-07-31')[['close']]* weight

        df['close'] += data['close']
    

    return (df, portfolio_stocks, weights)

In [63]:
(df, composition, weights) = create_portfolio()

# Metrics
- VAR
- RSI
- 252 SMA & 25 SMA

In [64]:
def var_historic(df, level=1):
    """
    Takes in a series of returns (r), and the percentage level
(level)
    Returns the historic Value at Risk at a specified level
    i.e. returns the number such that "level" percent of the returns
    fall below that number, and the (100-level) percent are above
    """
    r = df['close'].diff()

    r = r.dropna()
    if isinstance(r, pd.DataFrame):
        return r.aggregate(var_historic, level=level)
    elif isinstance(r, pd.Series):
        
        return -np.percentile(r, level)
    else:
        raise TypeError("Expected r to be a Series or DataFrame")

In [65]:
def rsi(df, periods = 14, ema = True):
    """
    Returns a pd.Series with the relative strength index.
    """
    close_delta = df['close'].diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    
    if ema == True:
        # Use exponential moving average
        ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    else:
        # Use simple moving average
        ma_up = up.rolling(window = periods, adjust=False).mean()
        ma_down = down.rolling(window = periods, adjust=False).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))
    return list(rsi)[-1]

In [66]:
def sma(df):
    sma1 = df['close'].rolling(window=42).mean()
    last1 = list(sma1)[-1]
    sma2 = df['close'].rolling(window=150).mean()
    last2 = list(sma2)[-1]
    return (last1, last2)

In [69]:
portfolios = pd.DataFrame([[composition, sma(df)[0], sma(df)[1], rsi(df), var_historic(df)]])

In [70]:
for i in range(0, 100, 1):
    (df, composition, weights) = create_portfolio()
    portfolios = portfolios.append(pd.DataFrame([[composition, sma(df)[0], sma(df)[1], rsi(df), var_historic(df)]]))

In [53]:
portfolios.reset_index(drop=True, inplace=True)

In [54]:
portfolios.columns = ['stocks', 'SMA42', 'SMA150', 'RSI', 'VAR']

ValueError: Length mismatch: Expected axis has 10 elements, new values have 5 elements

In [55]:
portfolios.to_csv('portfolios.csv')

In [56]:
portfolios

,0,1,2,3,4,RSI,SMA150,SMA42,VAR,stocks
0,NaN,NaN,NaN,NaN,NaN,33.671152,72.506717,73.350100,3.645800,"[VALE3, SUZB3, SULA11]"
1,NaN,NaN,NaN,NaN,NaN,43.200782,30.458006,32.261952,2.266972,"[HYPE3, BRDT3, PETR3, VIVT3, LCAM3]"
2,NaN,NaN,NaN,NaN,NaN,42.582970,22.933391,24.902040,2.046278,"[PETR3, BRFS3, IRBR3, PRIO3, IGTA3]"
3,NaN,NaN,NaN,NaN,NaN,31.714659,34.236428,33.783745,4.019889,"[CSNA3, SULA11, BBAS3]"
4,NaN,NaN,NaN,NaN,NaN,36.630423,41.760314,46.431161,2.382388,"[ITSA4, VIVT3, BIDI11]"
...,...,...,...,...,...,...,...,...,...,...
128,"[YDUQ3, PETR4, HYPE3, ENGI11, BRFS3, MGLU3, CS...",37.807724,34.012908,41.424997,2.520401,NaN,NaN,NaN,NaN,NaN
129,"[CSAN3, LREN3, FLRY3, ENGI11, BEEF3, CRFB3, AZ...",28.861202,28.427426,31.823131,2.914693,NaN,NaN,NaN,NaN,NaN
130,"[ITUB4, CCRO3, FLRY3, PCAR3, GOLL4, MRVE3, ENB...",20.599065,20.071782,27.022717,2.348812,NaN,NaN,NaN,NaN,NaN
131,"[LREN3, VVAR3, ENBR3, CRFB3, GGBR4, TAEE11, EL...",31.581796,28.631682,29.552641,2.985744,NaN,NaN,NaN,NaN,NaN
